# I made a little game for you:

In [ ]:
import viziometrics_chart_game as vcg

# Enjoy

In [ ]:
# 1.  set keywords to a topic that interests you:

vcg.set_keywords('uti')

In [ ]:
# 2.  set your username (optional)

# vcg.set_user('ITHS Seminar Participant')

In [ ]:
vcg.play()